In [2]:
import numpy as np
import tsplib95 as tsp

In [38]:
from decimal import localcontext, Decimal, ROUND_HALF_UP

In [6]:
from population import Population
from ga_operators import Selection_Tournament

## Classes

In [164]:
class TSP_Problem:
    '''
    The Djibouti TSP problem.

    '''
    def __init__(self, problem_name, cities_coords):
        '''
        expecting an array of cities with each
        city's (x,y) coordinate
        cities are numbered from 1 to DIM
        Dimension of the array is therefore (DIM, 2)
        '''
        self.type = "DISCRETE"
        self.dim = cities.shape[0]
        self.lbound, self.ubound = (1,self.dim)
        self.cities_coords = cities_coords
        self.problem_name = problem_name
        
    def get_distance_int(city1, city2):
        '''
        compute the euclidean distance (Norm2)
        between (city1, city2). The result is rounded
        half-up i.e. 2.5 -> 3
        Parameters
        ----------
        - city1, city2: The cities for which to compute
          the distance
        Return
        ------
        - the distance as an int
        '''
        distance = Decimal(np.linalg.norm(city1 - city2))
    
        # round half up
        with localcontext() as ctx:
            ctx.rounding = ROUND_HALF_UP
            distance = int(distance.to_integral_value())
        
        return int(distance)
    
    def get_distance(city1, city2):
        '''
        a "float variation" of the computation of the 
        euclidean distance (Norm2) between (city1, city2).
        The result is a rounded to 3 decimal places
        Parameters
        ----------
        - city1, city2: The cities for which to compute
          the distance
        Return
        ------
        - the distance as a numpy.float64
        '''
        distance = np.linalg.norm(city1 - city2)
    
        return round(distance, 3)

    def fitness(self, paths):
        '''
        Compute the fitness of matrix paths.
        Parameters
        ----------
        - paths:  a ndarray of dimension (population_size, dimension)
            containing the different paths (individuals in GA terms!)
            of the current generation
        Return
        ------
        - f:  fitness value(s) vector of dimension (dimension, ),
            representing the fitness of the paths of the current
            generation
        '''
        
        # a Vector of size (population_size, ) to hold
        # fitness values of the entire population (all paths)
        population_size = paths.shape[0]
        fitness = -1*np.ones(population_size)

        for idx in range(population_size):
            path = paths[idx]
            
            distance = 0
            for city1, city2 in zip(path, path[1:]):
                # cities in TSP files are numerated from
                # 1 to DIM. However self.cities being indexed
                # starting with 0, coordinate of city number 'i'
                # is at self.cities[i - 1]
                d = self.get_distance_int(self.cities_coords[city1_idx - 1],
                                          self.cities_coords[city2_idx - 1])
                distance = distance + d
            fitness[idx] = distance
        
        return(fitness)

## Functions
### Computing distance
Rounding half up ...

In [132]:
def get_distance_int(a,b):
    # compute the euclidean distance
    # Norm2
    distance = Decimal(np.linalg.norm(a-b))
    
    # round half up. I.e. 2.5 -> 3
    with localcontext() as ctx:
        ctx.rounding = ROUND_HALF_UP
        distance = distance.to_integral_value()
        
    return int(distance)

In [56]:
def get_distance(a,b):
    # compute the euclidean distance
    # Norm2
    distance = np.linalg.norm(a-b)
    
    return int(distance)

## Problem Quatar
**(Optimal tour for the qa194 TSP has length 9352)**. From http://www.math.uwaterloo.ca/tsp/world/qatour.html

In [148]:
quatar = tsp.load("./data/qa194.tsp")

## Problem (Djibouti)
**(optimal tour for the dj38 TSP has length 6656)**. From http://www.math.uwaterloo.ca/tsp/world/djtour.html
### Loading problem

In [57]:
djibouti = tsp.load("./data/dj38.tsp")

### Get x,y coordinates for all cities

In [146]:
djibouti_cities = np.array([djibouti.node_coords[city] for city in list(djibouti.get_nodes())])

In [147]:
djibouti_cities[:2]

array([[11003.6111, 42102.5   ],
       [11108.6111, 42373.8889]])

In [150]:
djibouti_pb = TSP_Problem("dj38", djibouti_cities)

'DISCRETE'

___

## TESTS

### Getting tsp file from internet

In [78]:
import requests
url = "http://www.math.uwaterloo.ca/tsp/world/dj38.tsp"
dj38 = requests.get(url)

In [84]:
dj38.content

"b'NAME: dj38\\nCOMMENT : 38 locations in Djibouti\\nCOMMENT : Derived from National Imagery and Mapping Agency data\\nCOMMENT : This file is a corrected version of dj89, where duplications\\nCOMMENT:  have been removed.  Thanks to Jay Muthuswamy and others for\\nCOMMENT:  requesting data sets without duplications.\\nTYPE: TSP\\nDIMENSION: 38\\nEDGE_WEIGHT_TYPE: EUC_2D\\nNODE_COORD_SECTION\\n1 11003.611100 42102.500000\\n2 11108.611100 42373.888900\\n3 11133.333300 42885.833300\\n4 11155.833300 42712.500000\\n5 11183.333300 42933.333300\\n6 11297.500000 42853.333300\\n7 11310.277800 42929.444400\\n8 11416.666700 42983.333300\\n9 11423.888900 43000.277800\\n10 11438.333300 42057.222200\\n11 11461.111100 43252.777800\\n12 11485.555600 43187.222200\\n13 11503.055600 42855.277800\\n14 11511.388900 42106.388900\\n15 11522.222200 42841.944400\\n16 11569.444400 43136.666700\\n17 11583.333300 43150.000000\\n18 11595.000000 43148.055600\\n19 11600.000000 43150.000000\\n20 11690.555600 42686.666

### Population and distance (fitness)

In [108]:
rng = np.random.default_rng()
dim = 38
population_size = 10
population = rng.choice(np.arange(1,dim+1), size=dim, replace=False)
for _ in range(population_size-1):
    population = np.concatenate((population,rng.choice(np.arange(1,dim+1), size=dim, replace=False)), axis=0)
population = population.reshape(-1,dim)

array([[ 2,  8,  5, 38, 31, 19, 17,  6,  9, 26, 22, 33, 28, 34, 30, 24,
         7, 18,  3, 12, 11, 20, 36, 15, 37,  4, 13, 29, 16, 35, 23, 25,
        21, 14, 10, 32, 27,  1],
       [12, 30,  4, 37, 15,  2, 34, 29, 32, 35, 18, 28, 26,  9, 33,  6,
        27, 16, 13, 14, 21, 36, 31, 22,  8, 17,  7, 10, 38, 23,  3,  5,
        20,  1, 24, 25, 19, 11],
       [18, 26,  1, 22, 29, 31, 37, 16, 12, 27, 14, 32,  2, 35,  3, 33,
        20, 25, 28, 34, 10,  4, 30, 23, 19, 21, 13,  6, 38,  9, 17, 15,
        11,  7, 24,  5, 36,  8],
       [20, 14, 18, 29,  1, 17, 32, 37, 12,  5, 35, 22, 25, 11, 10,  8,
         2, 23, 30, 15, 31, 36, 26,  7, 28,  9, 34, 16, 13,  6, 21,  4,
        38, 19, 24,  3, 33, 27],
       [32, 21, 33, 31, 34, 30, 25, 17, 37, 36,  4, 10,  2, 24,  7,  1,
         6, 28, 26, 13, 35,  8, 20, 22, 14, 18, 19,  3, 38, 12, 23, 27,
        29,  9, 16, 11, 15,  5],
       [ 9, 29,  7, 28, 14, 16,  6, 11, 34, 38, 12,  2, 27, 32, 17, 25,
        37, 18,  5, 15, 19,  4, 26, 35, 13,

In [160]:
dim = 38
population_size = 10
m = rng.choice(dim, size=dim, replace=False)
for _ in range(population_size-1):
    m = np.concatenate((m,rng.choice(dim, size=dim, replace=False)), axis=0)
m = m.reshape(-1,dim)

In [161]:
m

array([[ 6, 19,  4, 28, 15, 10,  7,  9, 29, 37, 33, 17, 18, 23, 11, 12,
        30, 25,  3, 31,  5, 27, 14,  2, 13, 36, 35, 24, 20, 26, 32,  1,
         0, 34, 16, 22, 21,  8],
       [33,  4,  6, 20, 37, 18, 17, 21, 31, 32,  5, 22,  3, 24, 26, 16,
         8,  7, 13, 19, 30,  2, 27, 34, 29,  1, 12, 14, 23,  9, 28,  0,
        15, 25, 36, 11, 10, 35],
       [13, 25,  9, 26, 37, 22,  1, 33,  8, 15, 19,  3, 23,  0, 36,  2,
        35, 11,  4, 24, 27, 29, 28,  6, 32, 31, 12,  7, 16,  5, 34, 20,
        10, 17, 14, 21, 18, 30],
       [ 5,  1,  7, 23, 27, 32, 36, 24,  3, 37, 19, 35,  4, 29, 10, 17,
         6, 30, 34, 31, 13,  9,  2,  8, 18, 16, 15, 22,  0, 11, 28, 26,
        14, 21, 33, 20, 25, 12],
       [ 9, 33, 11, 17, 36, 28,  6, 32,  7,  8, 37, 26,  0, 25, 12, 21,
        20, 27, 13, 29, 22,  5,  2, 10, 24, 31,  3, 35,  4, 16, 23, 19,
        18, 34, 30, 14, 15,  1],
       [ 3, 23, 35, 25,  5,  7, 12, 16, 26, 28, 14, 13, 18, 31,  2,  4,
        34,  1,  9,  8, 17, 37, 33,  6, 19,

In [163]:
np.sort(m[1])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37], dtype=int64)

In [128]:
cit = np.array(cities)
cit[0]

array([11003.6111, 42102.5   ])

In [159]:
rng.integers(1,30, size=10)

array([26,  1, 27, 15, 18,  9,  9, 15, 17, 24], dtype=int64)

### Euclidean norm computation

In [24]:
x1 = 11003.6111
y1 = 42102.5
x2 = 11108.6111
y2 = 42373.8889

In [25]:
d = np.sqrt(np.square(x1 - x2) + np.square(y1 - y2))
d

290.99301545433866

In [141]:
a = np.array([x1,y1])
b = np.array([x2,y2])
d = np.linalg.norm(a-b)
d

290.99301545433866

In [101]:
cit = np.array(cities)
cit.shape

(38, 2)

In [102]:
cit

array([[11003.6111, 42102.5   ],
       [11108.6111, 42373.8889],
       [11133.3333, 42885.8333],
       [11155.8333, 42712.5   ],
       [11183.3333, 42933.3333],
       [11297.5   , 42853.3333],
       [11310.2778, 42929.4444],
       [11416.6667, 42983.3333],
       [11423.8889, 43000.2778],
       [11438.3333, 42057.2222],
       [11461.1111, 43252.7778],
       [11485.5556, 43187.2222],
       [11503.0556, 42855.2778],
       [11511.3889, 42106.3889],
       [11522.2222, 42841.9444],
       [11569.4444, 43136.6667],
       [11583.3333, 43150.    ],
       [11595.    , 43148.0556],
       [11600.    , 43150.    ],
       [11690.5556, 42686.6667],
       [11715.8333, 41836.1111],
       [11751.1111, 42814.4444],
       [11770.2778, 42651.9444],
       [11785.2778, 42884.4444],
       [11822.7778, 42673.6111],
       [11846.9444, 42660.5556],
       [11963.0556, 43290.5556],
       [11973.0556, 43026.1111],
       [12058.3333, 42195.5556],
       [12149.4444, 42477.5   ],
       [12

In [ ]:
def ga(population, nb_generation, elite_ratio=0, 
       selection_op="tournament", selection_params={"K": 2},
       crossover_op="ordered", crossover_params={"crossover_proba": 0.9},
       mutation_op="swap", mutation_params={"mutation_proba": 0.1}):
    sel_op = 
    for _ in range(nb_generation):
        population.get_fitness()
        parents = 
        

In [4]:
p = My_Problem(10, (-10,10))

In [5]:
p.dim

10

In [6]:
p.lbound

-10

In [7]:
p.ubound

10

In [8]:
pop = Population(p, 100)

In [9]:
pop.individuals

array([[ 9.11356978e+00,  7.69216640e+00,  3.09634179e+00,
        -4.31873946e-01, -8.79325923e-01, -3.90266962e-01,
         3.72718173e+00,  2.82157944e+00, -7.36156284e+00,
         5.55328600e+00],
       [ 1.79822912e+00,  9.87704676e+00, -8.19012696e+00,
        -3.56787804e+00,  3.03323490e+00, -5.91915977e+00,
        -8.20281022e+00,  6.49661326e+00,  4.76439084e+00,
        -9.26365945e+00],
       [-8.87162935e+00,  9.92272326e+00,  8.05831785e-01,
        -2.46817434e+00,  1.18274828e+00, -2.63852175e+00,
         8.10164194e+00, -4.03917846e+00, -8.36881752e+00,
         1.53525089e-01],
       [-4.03408916e+00, -4.17703567e+00,  5.10342628e+00,
         7.02171524e+00,  4.08970968e-01,  8.79045721e+00,
        -3.51556209e+00,  7.91004881e+00, -4.37855808e+00,
         2.17540884e-01],
       [-4.93289672e+00,  4.24892010e+00,  4.23210419e+00,
         8.90455117e+00,  5.67856614e+00,  1.58095621e-01,
        -3.89911948e+00, -4.42721217e+00, -1.25475750e+00,
         7.

In [10]:
pop.get_fitness()

array([ 22.94109547,  -9.17411957,  -6.21985107,  13.34691437,
        15.94292426,  -7.96405867,  14.38118847,  -7.39759993,
        10.77499224,  -2.86435102,   5.47397318,  19.19317729,
       -11.44358794,  15.578942  ,  26.35958935,  -9.30733106,
       -17.91123105,  -3.28768988,   5.59734749, -34.2320834 ,
         2.34865828, -12.51673099, -15.96561485,  -8.4945969 ,
       -34.43135248,  30.41327115, -20.09297238,  17.40378385,
       -13.77977772,  -4.02574372,  21.54013233,  33.41168168,
         9.38855256, -42.48044958,  -4.74327924,   8.79233473,
       -11.8878271 , -16.78522478,   5.68325418,  15.34921838,
       -13.0021266 ,  16.68242818, -14.10074009, -15.06027259,
        10.63155909,  -7.29170474,  -5.97563506, -12.80909058,
       -26.13251191, -11.0421253 ,   0.54210921,  -8.98132444,
       -11.97526081, -23.82487768,   6.4226486 , -10.33690924,
        -2.87186769, -33.61428925,   2.00124084,  37.198036  ,
        15.7748056 ,  -4.91621752,  -0.26336774,  23.17

In [11]:
pop.fitness

array([ 22.94109547,  -9.17411957,  -6.21985107,  13.34691437,
        15.94292426,  -7.96405867,  14.38118847,  -7.39759993,
        10.77499224,  -2.86435102,   5.47397318,  19.19317729,
       -11.44358794,  15.578942  ,  26.35958935,  -9.30733106,
       -17.91123105,  -3.28768988,   5.59734749, -34.2320834 ,
         2.34865828, -12.51673099, -15.96561485,  -8.4945969 ,
       -34.43135248,  30.41327115, -20.09297238,  17.40378385,
       -13.77977772,  -4.02574372,  21.54013233,  33.41168168,
         9.38855256, -42.48044958,  -4.74327924,   8.79233473,
       -11.8878271 , -16.78522478,   5.68325418,  15.34921838,
       -13.0021266 ,  16.68242818, -14.10074009, -15.06027259,
        10.63155909,  -7.29170474,  -5.97563506, -12.80909058,
       -26.13251191, -11.0421253 ,   0.54210921,  -8.98132444,
       -11.97526081, -23.82487768,   6.4226486 , -10.33690924,
        -2.87186769, -33.61428925,   2.00124084,  37.198036  ,
        15.7748056 ,  -4.91621752,  -0.26336774,  23.17

In [ ]:
np.random.uniform(-10, 10, size=(20, 10))